# Lazy Evaluation

- There are occasions where some specific attributes have too large size, or take a really long time to compute
- In such cases, it may make sense to delay instantiating these attributes until they are truly needed
- How can we do this? We can make these attributes lazily evaluated; that is, they are only created when they are needed

- In the example here, we will implement lazy eval as a decorator
- There are 2 ways to write a decorator; either as a class itself, or as a function
    - We will demonstrate both in the example below
- The idea is the same though; you wrap something around an attribute, which delays the setting of the attribute until it is called
    - For the lazy_eval class, we rely on the custom `__get__` dunder to delay setting the attribute until it is called from the object
    - For the lazy_eval function, we rely on the `hasttr` and `setattr` methods to set the attribute once it has been called. Since the value is not in __init__, it is not set until later

In [30]:
from typing import Callable
from functools import update_wrapper

class LazyEvalProperty:
    def __init__(self, function: Callable):
        self.function: Callable = function
        update_wrapper(self, function)

    def __get__(self, obj, objtype=None):
        if obj is None:
            return self
        
        val = self.function(obj)
        obj.__dict__[self.function.__name__] = val
        return val

def lazy_eval_property(function: Callable):
    
    ## A property with name `SOMENAME` will look up the attribute _SOMENAME automatically
    ## Do ensure that there is an underscore prepended in the attr name below, or you will end up with an error
    attr: str = f"_{function.__name__}"

    @property
    def _lazy_property(self):
        if not hasattr(self, attr):
            setattr(self, attr, function(self))
        return getattr(self, attr)

    return _lazy_property

class TestClass:
    def __init__(self):
        self.attr1 = 123
        self.attr2 = 234

    @LazyEvalProperty
    def attr3(self):
        return 345
    
    @lazy_eval_property
    def attr4(self):
        return 456

tc = TestClass()
print(tc.__dict__)

tc.attr3
print(tc.__dict__)

tc.attr4
print(tc.__dict__)

{'attr1': 123, 'attr2': 234}
{'attr1': 123, 'attr2': 234, 'attr3': 345}
{'attr1': 123, 'attr2': 234, 'attr3': 345, '_attr4': 456}
